**What dataset are we working with?**

https://www.kaggle.com/datasets/mathurinache/citation-network-dataset/data?select=dblp.v12.json

How was the dataset collected?

For what purpose was the dataset collected?

In [1]:
import pandas as pd
import numpy as np
import json
import csv

from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile 


In [12]:
"""Run If You Don't Have The CSV"""

# Setup Kaggle API To Stream Dataset
api = KaggleApi()
api.authenticate()

api.dataset_download_files('mathurinache/citation-network-dataset')
print("dataset downloaded")
data_zf = ZipFile('citation-network-dataset.zip', 'r')
with data_zf:
    with data_zf.open('dblp.v12.json') as data:
        print("converting to df")
        json_data = json.loads(data.read())
        df = pd.DataFrame(json_data)

df.head()

Dataset URL: https://www.kaggle.com/datasets/mathurinache/citation-network-dataset
dataset downloaded
converting to df


### RUN clean_data.py OR GET data.csv FROM GOOGLE DRIVE BEFORE RUNNING THIS NOTEBOOK!

In [12]:

citations_df = pd.read_csv('indexed_data.csv')

citations_df.head()

SyntaxError: unterminated string literal (detected at line 1) (1090318074.py, line 1)

In [4]:
print(citations_df.columns)
print(citations_df.shape)

# Check for NA values
print(citations_df["ID"].isna().sum())

Index(['ID', 'Title', 'Year', 'Citations', 'Document Type', 'Authors', 'Venue',
       'Field of Study'],
      dtype='object')
(65563, 8)
0


There are 3,538,297 rows of data. Of these rows of data, 44 of them have an ID that is NA, so we have to drop these rows.

In [5]:
citations_df = citations_df.dropna(subset=["ID"])
citations_df = citations_df.dropna(subset=["Field of Study"])
print(citations_df.shape)

(65416, 8)


Check if all IDs are unique.

In [6]:
# Check if all IDs are unique
print(citations_df.shape[0])
print(len(citations_df["ID"].unique()))

ids = set()
repeatedIds = []
for id in citations_df["ID"]:
  if id in ids:
    repeatedIds.append(id)
  else:
    ids.add(id)

print("This is num of repeated IDs: " + str(len(repeatedIds)))
print("These are the repeated IDs:")

# The repeated Ids look weird. Some look like titles.
print(repeatedIds)
print(ids)

65416
65416
This is num of repeated IDs: 0
These are the repeated IDs:
[]
{68026369, 41811970, 37355523, 41418756, 20709385, 45350921, 58720268, 88604685, 76021774, 16121876, 44695576, 9044000, 30539812, 1966118, 62652455, 46792744, 2752553, 20971561, 39583785, 44040233, 38141997, 70778928, 15204401, 52953137, 37879859, 31981620, 74711088, 57802807, 35782712, 85983290, 83886139, 71696444, 82706493, 65011778, 20840515, 53346371, 5767238, 41025606, 68681801, 87687243, 73269325, 53608528, 34078801, 3276882, 12845138, 16777302, 10747992, 61735003, 21889116, 10747997, 33685597, 60162142, 65273949, 25559139, 30277732, 32768103, 89915495, 45744237, 30408814, 9699439, 60293235, 15859830, 23330934, 35651702, 68943992, 77332609, 61603971, 48496772, 18481288, 28967048, 57409673, 10616973, 49807502, 76284051, 52822164, 75104404, 73400471, 82051223, 69337243, 2621597, 21233822, 77856927, 8388768, 75628704, 63307938, 84934813, 9568420, 45088934, 13631661, 69075117, 15859887, 75235505, 89653425, 5636

There are 16 repeated IDs. All of which are titles. We will filter these rows out as well.

In [64]:
# Filter out rows with Document Type that don't look like document types i.e 'Conference', 'Journal', 'Repository', 'Patent', 'Book', nan
print(citations_df["Document Type"].unique())
docTypes =['Conference', 'Journal', 'Repository', 'Patent', 'Book']
# print(citations_df[citations_df["Document Type"].isin(docTypes)])


['Conference' 'Journal' 'Repository' nan 'Patent' 'Book'
 'Riccardo Caponetto, Viviana De Luca, Giovanna Di Pasquale, Salvatore Graziani, Francesca Sapuppo, Elena Umana'
 'Klaus Gürlebeck, Hung Manh Nguyen, Dmitrii Legatiuk' 'BookChapter'
 'Sudarshan K. Dhall, S. Lakshmivarahan, Chi-Ming Yang'
 'Bogdan Carbunar, Mahmudur Rahman, Jaime Ballesteros, Naphtali Rishe, Athanasios V. Vasilakos'
 'Beata Konikowska, Wojciech Penczek'
 'Kai-Yu Hu, Shih-Mei Lin, Chien-Hung Tsai'
 'Li Lu, Bozorgmehr Vosooghi, Liang Dai, Changzhi Li'
 'Qingyuan Zhang, E. M. Middleton, Bo-Cai Gao, Yen-Ben Cheng'
 'Mohammad Esmaeildoust, Dimitrios Schinianakis, Hamid Javashi, Thanos Stouraitis, Keivan Navi'
 'Omer Can Akgun, J. N. Rodrigues, Y. Leblebici, V. Owall'
 'Tim Morrison, Manohar Nagaraju, Brent Winslow, Amy Bernard, Brian P. Otis'
 'Jianxin Wu, Nini Liu, Christopher Geyer, James M. Rehg'
 'Fred T. Krogh, Kris Stewart' 'Kisoo Kim, Hokyu Lee, Chulwoo Kim'
 'Norio Yamazaki' 'Ja-Won Seo, Seong Dae Kim'
 'Hossei

In [6]:
docTypes =['Conference', 'Journal', 'Repository', 'Patent', 'Book']
# print(citations_df[citations_df["Document Type"].isin(docTypes)])
print(citations_df["Document Type"])


0      Conference
1         Journal
2      Conference
3      Conference
4      Repository
          ...    
653    Conference
654    Conference
655    Conference
656    Conference
657    Conference
Name: Document Type, Length: 658, dtype: object


In [7]:
print(pd.NA)

<NA>


In [8]:
# These columns are Series data types
# print(citations_df.Year.min())
# print(citations_df.Year.max())

# Look for papers whose year isn't 0
years_citations_df = citations_df[citations_df["Year"] != 0]
print(years_citations_df.shape)
print(years_citations_df.Year.min())
print(years_citations_df.Year.max())

(658, 8)
1969
2018


In [7]:
print(citations_df[citations_df["Year"] ==1])

Empty DataFrame
Columns: [ID, Title, Year, Citations, Document Type, Authors, Venue, Field of Study]
Index: []


There are no papers whose Year is 0.

IGNORE STUFF DOWN HERE FOR NOW

In [ ]:
# Papers written between 1800 (inclusive) and 1899 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 1800) & (years_citations_df["Year"] < 1900)])

In [ ]:
# Papers written between 1900 (inclusive) and 1999 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 1900) & (years_citations_df["Year"] < 1999)])

In [ ]:
# Papers written past 2000 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 2000)])

In [ ]:
# testingData = {
#     'Name': ['John', 'Doe', 'Alice', 'Bob', 'Chris'],
#     'Age': [25, 30, 22, 28, 35],
#     'Salary': [50000, 60000, None, 75000, 90000],
#     'Experience': [2, 5, 1, None, 10]
# }

# testing_df = pd.DataFrame(testingData)

# print(testing_df["Salary"].isna())
# print(testing_df["Salary"].isna().sum())

0    False
1    False
2     True
3    False
4    False
Name: Salary, dtype: bool
1


In [8]:
print(citations_df["ID"].isna().sum())
print(citations_df["Field of Study"].isna().sum())

print(len(citations_df.Title.unique()))
print("There are " + str(citations_df.shape[0] - len(citations_df.Title.unique())) + " papers that share the same title" )

0
0
65415
There are 1 papers that share the same title


Of the 4894081 papers in the dataset, 197111 papers share the same title.

In [9]:
print(citations_df.Citations.min())
print(citations_df.Citations.max())

print(citations_df[citations_df["Citations"] >= 48327])
print(citations_df[citations_df["Citations"] == 0])

1
3450
Empty DataFrame
Columns: [ID, Title, Year, Citations, Document Type, Authors, Venue, Field of Study]
Index: []
Empty DataFrame
Columns: [ID, Title, Year, Citations, Document Type, Authors, Venue, Field of Study]
Index: []


WORKING ON ISSUE #1

In [10]:
fosDict={}

for fields in citations_df["Field of Study"]:
  fieldsList=fields.split(", ")
  for field in fieldsList:
    # print(field)
    fosDict[field] = fosDict.get(field, 0) + 1
  # print("---------")

print(fosDict)


fieldsKeys=list(fosDict.keys())
fieldsValues=[fosDict.get(field) for field in fieldsKeys]

fos_df = pd.DataFrame({
    "Field": fieldsKeys,
    "Frequency": fieldsValues
})

fos_df.set_index("Field", inplace=True)

fos_df = fos_df.sort_values("Frequency", ascending=False)
print(fos_df.head(5))

{'Telecommunications network': 84, 'Computer science': 53134, 'Mind map': 2, 'Human–computer interaction': 2607, 'Multimedia': 2693, 'Empirical research': 398, 'Comprehension': 71, 'Communications protocol': 107, 'Graph': 930, 'Discrete mathematics': 3774, 'Combinatorics': 2059, 'Direct product': 9, 'Mathematics': 5351, 'Statue': 1, 'Engineering drawing': 242, 'Virtual reconstruction': 1, 'Visualization': 561, 'Polychrome': 2, 'Artificial intelligence': 17802, 'Autoregressive–moving-average model': 6, 'Support vector machine': 599, 'Neural network nn': 3, 'Financial time series prediction': 2, 'Autoregressive conditional heteroskedasticity': 13, 'Artificial neural network': 1102, 'Machine learning': 6824, 'Pattern recognition': 5057, 'Correlation attack': 16, 'Gross–Pitaevskii equation': 2, 'Hamiltonian (quantum mechanics)': 22, 'Mathematical analysis': 484, 'Schrödinger equation': 7, 'Periodic boundary conditions': 2, 'Hamiltonian mechanics': 1, 'Spectral method': 11, 'Wave equation':

In [11]:
# Explode the “Field of Study” column to create separate rows for each field
citations_df = citations_df.explode("Field of Study")

# Group citations dataset by the “Field of Study” column
grouped = citations_df.groupby("Field of Study")

# Initialize a dictionary to store the top 5 influential figures in each field
top_influential_figures = {}

# Iterate over each group
for field, group in grouped:
    # Initialize a dictionary to store the cumulative citations for each author
    author_citations = {}

    # Iterate over each row in the group
    for index, row in group.iterrows():
        # Split authors by comma and iterate over them
        authors = row["Authors"].split(", ")

        # Calculate cumulative citations for each author
        for author in authors:
            author_citations[author] = author_citations.get(author, 0) + row["Citations"]

    # Sort authors by their cumulative citations in descending order
    sorted_authors = sorted(author_citations.items(), key=lambda x: x[1], reverse=True)
    
    # Select the top 5 authors with the highest cumulative citations
    top_influential_figures[field] = sorted_authors[:5]